# Loopy - Filesystem over a String

A lightweight tree structure with Unix-like operations, stored as a single string.

Perfect for agents to manipulate structured data organically.

In [ ]:
from loopy import Loopy

## 1. Basics - It's Just a String

In [ ]:
# Start with an empty tree
tree = Loopy()
print("Empty tree:", tree.raw)

# Add some structure
tree.mkdir("/projects/webapp/src", parents=True)
tree.touch("/projects/webapp/src/main.py", "print('hello')")
tree.touch("/projects/webapp/README.md", "# My App")

print("\nWith content:", tree.raw)

In [ ]:
# Visualize it
print(tree.tree())

## 2. Chaining - It's Functional!

In [ ]:
# Chain everything - all mutations return self
knowledge = (
    Loopy()
    .mkdir("/facts/science/physics", parents=True)
    .mkdir("/facts/science/biology", parents=True)
    .mkdir("/facts/history", parents=True)
    .touch("/facts/science/physics/gravity", "9.8 m/s^2 on Earth")
    .touch("/facts/science/physics/light", "299,792,458 m/s")
    .touch("/facts/science/biology/dna", "double helix, ATCG bases")
    .touch("/facts/history/moon_landing", "July 20, 1969")
)

print(knowledge.tree())

In [ ]:
# Wild chaining: create, modify, reorganize in one flow
result = (
    Loopy()
    .touch("/inbox/task1", "Buy groceries")
    .touch("/inbox/task2", "Call mom")
    .touch("/inbox/task3", "Fix bug #123")
    .mkdir("/done", parents=True)
    .mv("/inbox/task2", "/done/task2")  # Complete a task
    .sed("/inbox/task3", "bug", "critical bug")  # Update priority
    .cp("/inbox/task1", "/reminders/groceries")  # Also remind me
)

print(result.tree())
print("\nUpdated task3:", result.cat("/inbox/task3"))

## 3. cd & Relative Paths

In [ ]:
# cd changes the working directory
project = (
    Loopy()
    .mkdir("/myapp/src/components", parents=True)
    .cd("/myapp")                     # now in /myapp
    .touch("README.md", "# My App")   # creates /myapp/README.md
    .touch("package.json", "{}")      # creates /myapp/package.json
    .cd("src")                        # relative cd -> /myapp/src
    .touch("index.js", "import App")  # creates /myapp/src/index.js
)

print(f"cwd: {project.cwd}")
print(f"ls(): {project.ls()}")      # lists current directory
print(f"ls('/'): {project.ls('/')}")  # lists root
print()
print(project.tree("/"))  # full tree from root

In [ ]:
logs = (
    Loopy()
    .touch("/logs/2024-01/error_001", "NullPointerException in UserService.java:42")
    .touch("/logs/2024-01/error_002", "ConnectionTimeout: database unreachable")
    .touch("/logs/2024-01/warn_001", "Deprecated API call in legacy module")
    .touch("/logs/2024-02/error_003", "OutOfMemoryError in ImageProcessor")
    .touch("/logs/2024-02/info_001", "Server started successfully")
)

print("All errors (by name):")
print(logs.grep("error"))

print("\nSearch content for 'database':")
print(logs.grep("database", content=True))

print("\nEverything NOT an error (invert):")
print(logs.grep("error", invert=True))

print("\nCount errors:")
print(logs.grep("error", count=True))

In [ ]:
# Regex power
print("Errors with numbers (regex):")
print(logs.grep(r"error_\d{3}"))

print("\nJava exceptions in content:")
print(logs.grep(r"\w+Exception", content=True))

## 4. Sed - Transform Content

In [ ]:
config = (
    Loopy()
    .touch("/config/dev", "DB_HOST=localhost;DB_PORT=5432;DEBUG=true")
    .touch("/config/staging", "DB_HOST=localhost;DB_PORT=5432;DEBUG=true")
    .touch("/config/prod", "DB_HOST=localhost;DB_PORT=5432;DEBUG=true")
)

print("Before:")
print(config.cat("/config/prod"))

# Update prod config
config.sed("/config/prod", "localhost", "prod-db.aws.com")
config.sed("/config/prod", "DEBUG=true", "DEBUG=false")

print("\nAfter:")
print(config.cat("/config/prod"))

In [ ]:
# Recursive sed - update ALL configs at once
config.sed("/config", "5432", "5433", recursive=True)

print("All configs after recursive sed:")
for env in config.ls("/config"):
    print(f"  {env}: {config.cat(f'/config/{env}')}")

In [ ]:
# Backreferences - capture and transform
data = Loopy().touch("/data", "user_123 user_456 user_789")

# Transform user_XXX to USER[XXX]
data.sed("/data", r"user_(\d+)", r"USER[\1]")
print(data.cat("/data"))

## 5. Glob & Find - Pattern Matching

In [ ]:
project = (
    Loopy()
    .touch("/src/main.py", "# main")
    .touch("/src/utils.py", "# utils")
    .touch("/src/models/user.py", "# user model")
    .touch("/src/models/post.py", "# post model")
    .touch("/tests/test_main.py", "# tests")
    .touch("/tests/test_utils.py", "# tests")
    .touch("/docs/README.md", "# readme")
    .touch("/docs/API.md", "# api docs")
)

print("All .py files (glob **/*.py):")
print(project.glob("/**/*.py"))

print("\nDirect children of /src (glob /src/*):")
print(project.glob("/src/*"))

print("\nAll .md files:")
print(project.glob("/**/*.md"))

In [ ]:
# Find with type filter
print("Directories only:")
print(project.find("/", type="d"))

print("\nFiles only:")
print(project.find("/", type="f"))

print("\nFind by name pattern (test_*):")
print(project.find("/", name=r"test_"))

## 6. Wild Combinations - Pipe-like Patterns

In [ ]:
# Simulate: grep + sed (find and replace across matching files)
codebase = (
    Loopy()
    .touch("/src/api.py", "import old_lib; old_lib.connect()")
    .touch("/src/db.py", "import old_lib; old_lib.query()")
    .touch("/src/utils.py", "import new_lib; new_lib.helper()")
)

# Find all FILES using old_lib (filter with find to get only leaves)
all_files = codebase.find("/src", type="f")
matches = [f for f in all_files if "old_lib" in codebase.cat(f)]
print(f"Files using old_lib: {matches}")

for path in matches:
    codebase.sed(path, "old_lib", "new_lib")
    print(f"  Migrated: {path} -> {codebase.cat(path)}")

In [ ]:
# Simulate: find + xargs + operation (batch processing)
media = (
    Loopy()
    .touch("/photos/vacation/img001.jpg", "size:2MB")
    .touch("/photos/vacation/img002.jpg", "size:3MB")
    .touch("/photos/work/screenshot.png", "size:500KB")
    .touch("/videos/clip.mp4", "size:50MB")
)

# Find all .jpg files and "compress" them (update metadata)
jpgs = media.glob("/**/*.jpg")
print(f"Found {len(jpgs)} JPG files")

for jpg in jpgs:
    media.sed(jpg, r"size:(\d+)MB", r"size:\1MB_compressed")

print("\nAfter 'compression':")
for jpg in jpgs:
    print(f"  {jpg}: {media.cat(jpg)}")

In [ ]:
# Build a processing pipeline
def process_pipeline(tree, path):
    """Chain: find python files -> check for TODO -> report"""
    py_files = tree.glob(f"{path}/**/*.py")
    todos = []
    for f in py_files:
        content = tree.cat(f)
        if "TODO" in content:
            todos.append((f, content))
    return todos

code = (
    Loopy()
    .touch("/project/main.py", "# TODO: add logging")
    .touch("/project/utils.py", "# helper functions")
    .touch("/project/api.py", "# TODO: add auth; TODO: rate limit")
)

todos = process_pipeline(code, "/project")
print("Files with TODOs:")
for path, content in todos:
    print(f"  {path}: {content}")

## 7. Agent Use Case - Organic Knowledge Building

In [ ]:
# Simulate an agent categorizing images it sees
import random

knowledge = Loopy()

# Agent "sees" images and categorizes them
observations = [
    ("golden retriever playing", "/animals/dogs/golden_retriever"),
    ("persian cat sleeping", "/animals/cats/persian"),
    ("tesla model 3", "/vehicles/cars/electric/tesla"),
    ("german shepherd", "/animals/dogs/german_shepherd"),
    ("oak tree in park", "/plants/trees/oak"),
    ("siamese cat", "/animals/cats/siamese"),
    ("ford f150 truck", "/vehicles/trucks/ford_f150"),
    ("rose garden", "/plants/flowers/rose"),
]

for description, category in observations:
    # Check if similar category exists
    parent = "/".join(category.rsplit("/", 1)[:-1])
    
    # Agent decides to store it
    if not knowledge.exists(category):
        knowledge.touch(category, description)
        print(f"Learned: {category}")

print("\n" + knowledge.tree())

In [ ]:
# Agent queries its knowledge
print("What dogs do I know about?")
dogs = knowledge.ls("/animals/dogs")
for dog in dogs:
    print(f"  - {dog}: {knowledge.cat(f'/animals/dogs/{dog}')}")

print("\nSearch for 'cat':")
print(knowledge.grep("cat"))

print("\nAll animals (recursive):")
print(knowledge.find("/animals", type="f"))

In [ ]:
# Agent reorganizes knowledge
print("Before reorganization:")
print(knowledge.tree())

# Realize we should group all pets together
knowledge.mkdir("/pets", parents=True)
knowledge.cp("/animals/dogs", "/pets/dogs")
knowledge.cp("/animals/cats", "/pets/cats")

print("\nAfter adding /pets category:")
print(knowledge.tree())

## 8. Serialization - Save & Load

In [ ]:
# The entire knowledge graph is just a string!
serialized = knowledge.raw
print(f"Serialized length: {len(serialized)} characters")
print(f"Preview: {serialized[:100]}...")

# Save to "database" (just a variable here)
database = serialized

# Later: restore from database
restored = Loopy(database)
print(f"\nRestored tree has {restored.du('/')} nodes")
print(restored.ls("/"))

## 9. Statistics & Inspection

In [ ]:
# du - disk usage
print(f"Total nodes: {knowledge.du('/')}")
print(f"Nodes under /animals: {knowledge.du('/animals')}")
print(f"Total content bytes: {knowledge.du('/', content_size=True)}")

# info - node metadata
print(f"\nInfo about /animals:")
print(knowledge.info("/animals"))

print(f"\nInfo about a leaf node:")
print(knowledge.info("/animals/dogs/golden_retriever"))

In [ ]:
# walk - os.walk() style traversal
print("Walking the tree:")
for dirpath, dirs, files in knowledge.walk("/animals"):
    print(f"  {dirpath}/")
    print(f"    dirs: {dirs}")
    print(f"    files: {files}")

## 10. Edge Cases & Special Content

In [ ]:
# Content with special characters (auto-escaped)
special = Loopy()
special.touch("/html", "<div class='test'>Hello & goodbye</div>")
special.touch("/math", "x < y && y > z")
special.touch("/code", "if (a < b) { return a & b; }")

print("Content is properly escaped and unescaped:")
print(f"  /html: {special.cat('/html')}")
print(f"  /math: {special.cat('/math')}")
print(f"  /code: {special.cat('/code')}")

print(f"\nRaw storage (escaped):")
print(special.raw)

In [ ]:
# Unicode works great
unicode_tree = (
    Loopy()
    .touch("/greetings/english", "Hello World!")
    .touch("/greetings/japanese", "こんにちは世界")
    .touch("/greetings/emoji", "Hello! ")
    .touch("/greetings/arabic", "مرحبا بالعالم")
)

print(unicode_tree.tree())
for lang in unicode_tree.ls("/greetings"):
    print(f"{lang}: {unicode_tree.cat(f'/greetings/{lang}')}")

## 11. One-Liner Madness

In [ ]:
# Build, query, and transform in one expression
result = [
    path 
    for path in Loopy()
        .touch("/a/x", "hello")
        .touch("/a/y", "world")
        .touch("/b/z", "test")
        .find("/", type="f")
    if "a" in path
]
print("Leaf nodes under /a:", result)

## Summary

Loopy gives you:
- **Filesystem semantics** over a Python string
- **Functional chaining** - all operations return `self`
- **cd & relative paths** - work in context
- **Full regex support** in grep, sed, find
- **Glob patterns** for path matching
- **Zero dependencies** - just Python stdlib
- **~500 lines** of code

Perfect for agents building organic knowledge structures!

## Summary

Loopy gives you:
- **Filesystem semantics** over a Python string
- **Functional chaining** - all operations return `self`
- **Full regex support** in grep, sed, find
- **Glob patterns** for path matching
- **Zero dependencies** - just Python stdlib
- **~450 lines** of code

Perfect for agents building organic knowledge structures!